# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

from BB_agent_tool import BB_agent_tool

import pdb

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 06645787-20fd-7e61-8000-bc9909f8fdc8
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'fred_api_key': SecretStr('**********'), 'biztoc_api_key': SecretStr('**********'), 'benzinga_api_key': None, 'alpha_vantage_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'nasdaq_api_key': None, 'tradier_api_key': None, 'tradier_account_type': None, 'polygon_api_key': SecretStr('**********'), 'intrinio_api_key': None, 'tradingeconomics_api_key': None, 'tiingo_token': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 06645787-2541-7787-8000-db98d5b72359
results: [{'symbol': 'MRK', 'one_day': 0.0239, 'wtd': None, 'one_week': 0.0168, 'mt...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=0.0239, wtd=None, one_week=0.0168, mtd=None, one_month=0.0533, qtd=None, three_month=0.047, six_month=0.28550000000000003, ytd=0.20829999999999999, one_year=0.132, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0134, volatility_month=0.0149, price=131.73, volume=7224765.0, average_volume=8210000.000000001, relative_volume=0.88, analyst_recommendation=None, analyst_score=1.48)]

In [13]:
obj.dict()


{'id': '06645787-2541-7787-8000-db98d5b72359',
 'results': [{'symbol': 'MRK',
   'one_day': 0.0239,
   'wtd': None,
   'one_week': 0.0168,
   'mtd': None,
   'one_month': 0.0533,
   'qtd': None,
   'three_month': 0.047,
   'six_month': 0.28550000000000003,
   'ytd': 0.20829999999999999,
   'one_year': 0.132,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0134,
   'volatility_month': 0.0149,
   'price': 131.73,
   'volume': 7224765.0,
   'average_volume': 8210000.000000001,
   'relative_volume': 0.88,
   'analyst_recommendation': None,
   'analyst_score': 1.48}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 153264250,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 15, 23, 7, 3

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 131.73,
   'bid_size': 800,
   'ask': 131.68,
   'ask_size': 800,
   'last_price': 131.73,
   'open': 128.88,
   'high': 131.965,
   'low': 128.72,
   'volume': 7223237,
   'prev_close': 128.66,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.818,
   'ma_200d': 114.56235,
   'volume_average': 8261309.0,
   'volume_average_10d': 6897520.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 170379000,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-15T23:07:30.348393'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-15,117.14,117.740,115.4900,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.2900,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.4800,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.3300,114.00,6437144.0,114.0864,75952
2023-05-19,114.33,116.240,114.1000,115.49,7647080.0,115.4905,63052
...,...,...,...,...,...,...,...
2024-05-08,130.58,131.510,129.3300,129.55,6580073.0,129.7913,74021
2024-05-09,128.94,130.500,128.9400,130.23,9038061.0,129.9542,70936
2024-05-10,130.82,130.880,129.9500,130.06,5540149.0,130.3223,73151


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
2023-05-19,418.62,420.7200,417.350,420.17,642871,103793317.0,419.0232
...,...,...,...,...,...,...,...
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211
2024-05-09,520.17,520.2074,516.705,517.38,399512,43583253.0,519.1084
2024-05-10,520.84,522.6350,519.590,521.81,428536,52201942.0,520.8834


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.89,2024-05-15 20:00:02+00:00,56.90,57.12,56.52,1821579,56.72,...,66.15,54.12,58.6438,60.15885,3059839,1.008148e+11,1772100000,4.61,12.34,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,78.68,2024-05-15 20:00:02+00:00,77.42,78.79,77.38,6369948,77.07,...,78.79,48.66,72.0174,63.23660,7058424,1.398364e+11,1777280000,2.39,32.92,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,202.11,2024-05-15 20:00:02+00:00,202.01,202.69,199.77,8115081,201.51,...,202.69,134.40,192.7314,166.44000,8847162,5.803932e+11,2871670000,16.56,12.20,2024-07-12 00:00:00+00:00
3,MS,Morgan Stanley,NYSE,100.52,2024-05-15 20:00:47+00:00,99.00,100.98,99.00,6918368,99.38,...,100.98,69.42,91.6292,85.79915,7980975,1.633611e+11,1625160000,5.50,18.28,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-13 15:16:00+00:00,Bristol Myers (BMY) Fails to Meet Goal in Opdi...,Bristol Myers' (BMY) late-stage label-expandin...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272954/brist...,"AZN,BMY,MRK,LGND",Zacks Equity Research,DZ7EFMYgND4Mcq1fh_0B9LZ6EWP-9-JQz_pR6koaUmA,https://www.zacks.com/amp/stock/news/2272954/b...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-14 13:25:00+00:00,Merck (MRK) Ends Keytruda Combo Melanoma Study...,Merck (MRK) discontinues the Keytruda plus vib...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2273402/merck...,"MRK,MRNA,LGND,ANIP",Zacks Equity Research,R1VrrdEUIYHeJqiOWSzIxRUOlqB_Oa1pGnAYJYBm06w,https://www.zacks.com/amp/stock/news/2273402/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 05:22:49+00:00,MannKind Outperforms As Tyvaso DPI Captures Ma...,MannKind's long-term liabilities exceed its as...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4693345-mannk...,"LQDA,MRK,UTHR,MNKD",Stephen Ayers,fQ4xeEwbTVYO0LoI2YgpHdzGmeDLS3cKjfUdvyuUBfA,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 12:45:36+00:00,Pfizer: Best Trading Momentum Since 2021 And 6...,Pfizer has been a losing investment for severa...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4693405-pfize...,"ABBV,AMGN,AZN,AZNCF,BMY,BMYMP,GILD,JNJ,LLY,MRK...",Paul Franke,q9bmkpb18oxPdfIlrtS-uA0trrZ-yMQWBaO6f28ZfkM,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 17:11:00+00:00,"Top Stock Reports for Broadcom, Merck & Airbnb",Today's Research Daily features new research r...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/research-daily/2274193/t...,"PCG,MRK,OKE,ENB,AVGO,ABNB",Sheraz Mian,m4MxY768tHMwb6pp0h-AmvZU_TduiMvnjVNHpoVNjDQ,None,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [21]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
# utility function to call chatgpt
def get_response(messages, tools, model=MODEL, json_format=False):
    """
    Get a single response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.
        json_format(boolean): True if JSON response requested. (Last message must express the request for JSON response.)

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # can't pass None, need to pass NotGiven
        tools=tools if tools else openai.NotGiven(),
        # tool_choice="auto",  # auto is default, but we'll be explicit
        response_format={"type": "json_object"} if json_format else None,
    )

    return response


In [23]:
client = OpenAI()
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The question of the airspeed velocity of an unladen swallow famously comes from the comedy film "Monty Python and the Holy Grail." In the film, the characters are referring to the European swallow (Hirundo rustica). 

However, to provide a more scientific answer, the airspeed of an unladen European swallow has been estimated to be around 11 meters per second, or roughly 24 miles per hour. This figure comes from ornithological studies that have measured the flight speed of various bird species.

It is also important to note that there are many different species of swallows, and their flight speeds can vary. For example, the African swallow (Hirundo smithii) might have different flight characteristics.

So, the airspeed velocity of an unladen European swallow is approximately 11 meters per second.


# Question Answering Agent

In [24]:
# utility functions to support use of client-side tools when querying openai

def eval_tool(tool_call, verbose=True):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    try:
        function_name = tool_call.function.name
        # look up the function based in global tools on the name
        fn = BB_agent_tool.agent_registry[function_name]
        # make the tool call's json args into a dict
        kwargs = json.loads(tool_call.function.arguments)
        
        if verbose:
            print(f"{function_name}({str(kwargs)}) -> ", end="")
        # call function with the args and return value
        fn_value = fn(**kwargs)
        if type(fn_value) is list or type(fn_value) is dict:
            fn_value=str(fn_value)
        if verbose:
            output = str(fn_value)
            if len(output) > 100:
                output=output[:100] + "..."
            print(output)
    
        return {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_call.function.name,
            "content": fn_value,
        }
    except Exception as exc:
        return f"Error: {exc}"

def get_response_and_eval(messages, tools=[], json_format=False, raw=False, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the response.
    If tool calls are returned, calls all the tools and sends the values back to OpenAI.
    If further tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        json_format (boolean): If the final response should be in JSON format.
        raw (boolean): after last tool is called return raw data response that enabled answering the question
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The final response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response(messages, tools=tools, json_format=json_format)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            for tool in tool_calls:
                messages.append(eval_tool(tool, verbose=True))
            # Get next response
            response = get_response(messages, tools=tools, json_format=json_format)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                output = str(choice)
                output = output[:1000] + "..." if len(output) > 1000 else output
                print(output)
        else:
            print('finish_reason: ', finish_reason)
            break

    if raw:
        # probably want to process that message and return call signature + value
        return messages[-1]
    else:
        return response

def agent_query(user_message, raw=False, verbose=True):
    """
    Send a user message to OpenAI and retrieve the response, calling all tools until done.

    Args:
        user_message (str): The message from the user.
        raw (boolean): after last tool is called return raw data response that enabled answering the question        
        verbose (bool, optional): Display intermediate tool calls and return values. Defaults to False.

    Returns:
        str: The response from the agent.


    Example:
        >>> agent_query("Hello")
        'Hello! How can I assist you today?'
    """

    # recompute system prompt, adding tool metadata i.e. descriptions of available tools to system prompt
    tool_descs = ""
    openai_tools = []
    for v in tool.agent_registry.values():
        t = v.tooldict
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        if v.example_code:
            tool_descs += f" Usage: {v.example_code}"
        tool_descs += "\n---\n"
    # tool_descs = "\n".join([f"{tool['function']['name']} : {tool['function']['description']}" for tool in tools.values()])
    current_system_prompt = system_prompt + f"""

Available tools, with name, description, and calling example, delimited by ---:
{tool_descs}
    """
    # print(current_system_prompt)
    RETRIES = 3
    for retry in range(RETRIES):
        try:
            if retry:
                print(f"retrying, attempt {retry + 1}")
            messages = [{"role": "system", "content": current_system_prompt},
                        {"role": "user", "content": user_message}]
            response = get_response_and_eval(messages, tools=openai_tools, json_format=False, raw=raw, verbose=verbose)
            response_str = response.choices[0].message.content
            response_str = response_str.replace("$", "\\\$")   # escape stuff that is interpreted as latex
            display(Markdown(response_str))
            # success, exit retry loop
            break
        except Exception as exc:
            print(exc)



In [25]:
# make a bespoke tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = json.loads(tool(symbol="MSFT"))
len(tool_response[0]['item1'])

# create tool dict based on metadata
# def make_tool_dict(fn_metadata):
#     retdict  = {
#         "type": "function",
#         "function": {
#             "name": fn_metadata["name"],
#             "description": fn_metadata["description"],
#             "parameters": {
#                 "type": "object",
#                 "properties": fn_metadata["parameters"],
#                 "required": [k for k in fn_metadata["parameters"].keys()]
#                 }
#             }
#         }
#     return retdict

# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


143226

In [26]:
# add some stuff about first figure out if it there is a ticker or a company, if not then exit
# if confusion about whether it is a ticker or company then exit
system_prompt = """
Role: You are an AI stock market assistant tasked with providing up-to-date, 
detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate, 
complete, but concise answers relevant to their questions about individual 
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools 
as needed to ensure all information provided is timely, accurate, concise, 
relevant, and responsive to the user's query.

Instructions: 
1. Input validation. Determine if the input is asking about a specific company 
or stock ticker. If not, respond that you are unable to answer and suggest 
alternative information sources.

2. Entity extraction. If the query is valid, extract the company name or ticker 
symbol from the question. If a company name  is given, look up the ticker symbol
using a tool. If the ticker symbol is not found, you may try to correct the 
spelling and try again, like changing "microsfot" to "microsoft", or changing
"southwest airlines" to a simpler variation like "southwest" and increasing 
"limit" to 10. If the company or 
ticker is unclear based on the question or conversation so far, and the results 
of the symbol lookup, then ask the user to clarify.

3. Information retrieval. Use the appropriate tools to fetch the requested 
information. If necessary, sequentially employ multiple tools—for instance, 
first determining the company's ticker, then retrieving the company data using 
the ticker.

4. Compose Response. Provide the answer to the user in a clear and concise format.

Example Interaction:
User asks: "What is the PE ratio for Eli Lilly?"
Chatbot recognizes 'Eli Lilly' as a company name.
Chatbot uses symbol lookup to find the ticker for Eli Lilly.
Chatbot retrieves the PE ratio using the proper function.
Chatbot responds: "The PE ratio for Eli Lilly (symbol: LLY) as of May 12, 2024 is 30."

Check carefully and only call the tools which are specifically named below, and take care to only use data obtained from these tools.

"""


In [27]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, verbose=False)


get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...


### Summary of Item 1 from Microsoft’s Latest 10-K Report

#### Business Overview
- **Microsoft's Mission**: To empower every person and organization on the planet to achieve more, creating local opportunity, growth, and impact globally. 
- **Technological Focus**: Developing platforms and tools powered by AI to support small and large businesses, improve educational and health outcomes, enhance public sector efficiency, and empower human ingenuity.

#### Market and Products
- **Products and Services**: This includes cloud-based solutions, operating systems, productivity and collaboration applications, server applications, and video games. Examples are Office 365, Dynamics 365, LinkedIn, Azure, GitHub, Surface devices, Xbox, and Bing.
- **Key Objectives**:
  - Reinvent productivity and business processes.
  - Build the intelligent cloud and intelligent edge platform.
  - Create more personal computing.

#### Technological Innovation
- **AI Investments**: Microsoft is incorporating AI across its offerings, including productivity tools like Microsoft Teams, Outlook, Bing, and Xbox. AI is positioned at the core of their innovative strategies.
- **Commitment to Innovation and Security**: Investing in AI development is accompanied by principles focusing on fairness, reliability and safety, privacy and security, inclusiveness, transparency, and accountability.

#### Corporate Responsibility
- **Sustainability Goals**: Committed to being carbon negative, water positive, and zero waste by 2030. Initiatives include power purchase agreements for green energy, water replenishment projects, and waste diversion programs.
- **Racial Equity Initiative**: A multi-year plan focused on strengthening communities, engaging the ecosystem, and increasing representation, particularly for Black, African American, Hispanic, and Latinx communities within the U.S.
- **Digital Skilling**: Programs aimed at preparing 250,000 U.S. workers for cybersecurity roles by 2025, and offering generative AI learning opportunities globally.

#### Human Capital
- **Employee Focused Culture**: Grounded in a growth mindset, promoting inclusiveness and innovation, and offering substantial training and benefits. Aims to ensure fair pay and career growth for diverse employee groups.

#### Operating Segments
- **Productivity and Business Processes**: Products like Office, LinkedIn, and Dynamics.
- **Intelligent Cloud**: Includes Azure, SQL Server, GitHub, and Nuance.
- **More Personal Computing**: Encompasses Windows, Surface devices, Xbox, and Bing.

#### Key Risks
- **Competition**: Intense competition across all markets might lead to lower revenue or operating margins.
- **Cybersecurity and Data Privacy**: Security vulnerabilities and cyberattacks pose risks to both Microsoft and its customers.
- **Innovation Risks**: Investments in new technologies may not achieve expected returns.
- **Compliance**: Increasing legal and regulatory requirements.
- **Intellectual Property**: Risks related to protecting and utilizing IP.

This summary highlights the key points from Microsoft's latest annual 10-K filing. It covers their mission, market focus, commitment to innovation, social responsibility initiatives, employee culture, operating segments, and key risk factors.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [28]:
# TODO:

# move object to a file , move llm calls to a file

# make a streamlit app that maintains conversation

# use assistants API


In [29]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [30]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [31]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [32]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)


In [33]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[{"symbol": "LUV", "name": "SOUTHWEST AIRLS CO COM", "dpm_name": "Susquehanna Securities, LLC", "post_station": "5/1"}]'

In [34]:
agent_query("What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest', 'limit': 1}) -> [{"symbol": "CSWC", "name": "CAPITAL SOUTHWEST CORP COM", "dpm_name": "Wolverine Trading, LLC", "pos...


The stock symbol for Southwest Airlines is LUV.

In [35]:
agent_query("What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 1}) -> []


I couldn't find any information on "yakimee air." It might be a spelling error or not a publicly traded company. Could you please provide more details or check the spelling?

In [36]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query("What is item 1 from the 10k annual report for Microsfot?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ABI4hftfEdnVgKyfF6OzZbca', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_ABI4hftfEdnVgKyfF6OzZbca', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'query': 'Microsoft', 'limit': 1}) -> [{"symbol": "MSFT", "name": "MICROSOFT CORP COM", "dpm_name": "Susquehanna Securities, LLC", "post_s...
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_248hlqL6Y4NjqSB8qXJ9gzxt', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='funct

Here is Item 1 from Microsoft's latest 10-K annual report:

**Business Overview**

**General**
Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more. It focuses on creating opportunities, growth, and impact globally. Microsoft develops software, services, devices, and solutions to help customers achieve full potential and improve various business functions through innovative technologies such as AI and cloud platforms.

**What We Offer**
Founded in 1975, Microsoft offers a variety of software, services, solutions, and devices aimed at creating value for its customers. This includes cloud-based solutions, operating systems, productivity and collaboration applications, server applications, business solutions, software development tools, and video games. The company also designs and sells devices such as PCs, tablets, gaming consoles, and accessories.

**Ambitions**
1. Reinvent productivity and business processes
2. Build the intelligent cloud and intelligent edge platform
3. Create more personal computing

**Focus Areas**
- Empowering employees with AI-backed tools
- Enabling compliance and data protection
- Innovating AI uses for businesses and individuals
- Providing comprehensive security, compliance, and identity solutions
- Expanding gaming experiences across various devices and platforms

**Corporate Social Responsibility**
Microsoft is committed to sustainability, addressing climate change, enhancing social equity, and investing in digital skills. Initiatives include carbon neutrality by 2030, racial equity programs, community engagement, and the development of skilling resources.

**Human Capital Resources**
As of June 30, 2023, Microsoft employed approximately 221,000 full-time employees worldwide, focusing on growth mindset, diversity and inclusion, and creating a rewarding work environment.

For more details, please refer to Microsoft's complete 10-K annual report.

In [37]:
agent_query(f"What is the last market quote for symbol {symbol}?", verbose=False)


I do not have direct access to live market quotes. For real-time or latest market quotes for Merck & Co., Inc. (symbol: MRK), I recommend checking financial news websites, stock market apps, or your brokerage's online platform. You can also visit platforms like Yahoo Finance, Bloomberg, or Google Finance for up-to-date information.

In [38]:
agent_query(f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 1}) -> []


I wasn't able to find a stock symbol for "Consolidated Agglomerators." Please make sure that the company name is spelled correctly or provide more details. If you have an alternative company or ticker symbol in mind, please let me know.

In [39]:
agent_query(f"What is the current weather in Las Vegas", verbose=False)


I'm sorry, but I can only assist with questions related to individual stocks and companies. For current weather information in Las Vegas, I recommend checking a weather service website or app, such as Weather.com or the Weather Channel. You can also use a search engine for the latest updates.

In [40]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Merck & Co Inc (Symbol: MRK)

- **Stock Exchange:** NYSE
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Description:** Merck & Co., Inc. is a health care company that provides health solutions through prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. The company operates through three segments: Pharmaceutical, Animal Health, and Other.
  - **Pharmaceutical:** Includes human health pharmaceutical and vaccine products.
  - **Animal Health:** Focuses on the development and marketing of pharmaceutical and vaccine products for livestock and companion animal species.
  - **Other:** Consists of sales from non-reportable healthcare service segments.
- **Headquarters:** Rahway, NJ, USA
- **Founded:** 1891
- **Employees:** 72,000
- **Market Capitalization:** \\$333.65 billion
- **Index Membership:** DJIA, S&P 500
- **Shares Outstanding:** 2.53 billion
- **Shares Float:** 2.53 billion
- **Short Interest:** 18.75 million
- **Institutional Ownership:** 79.05%
- **Beta:** 0.41

If you need more detailed information or specifics related to Merck & Co, feel free to ask!

In [41]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


Here are the latest statistics on short interest for Merck & Co., Inc. (symbol: MRK):

- **Settlement Date:** April 30, 2024
- **Current Short Position:** 18,754,018 shares
- **Previous Short Position:** 20,429,972 shares
- **Average Daily Volume:** 7,551,651 shares
- **Days to Cover:** 2.48 days
- **Change in Short Position:** -1,675,954 shares
- **Percentage Change:** -8.2%

If you need more detailed information or historical data, feel free to ask!

In [42]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [43]:
agent_query(f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


The historical stock splits for Apple Inc. (AAPL) are as follows:
1. **August 31, 2020:** 4-for-1
2. **June 9, 2014:** 7-for-1
3. **February 28, 2005:** 2-for-1
4. **June 21, 2000:** 2-for-1
5. **June 16, 1987:** 2-for-1

In [44]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The latest total assets for Merck & Co. (symbol: MRK) as of the fiscal year ending December 31, 2023, are \\$106.675 billion. 

You can find more detailed information in their [10-K report](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [45]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [46]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (MRK) for the fiscal year ending December 31, 2023, was \\$13,006,000,000 (USD).

For more detailed financial data, you can refer to their [latest 10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [47]:
tool.agent_registry

{'get_10k_item1_from_symbol': <BB_agent_tool.BB_agent_tool at 0x31a7bbb50>,
 'get_equity_search_symbol': <BB_agent_tool.BB_agent_tool at 0x31cabc1d0>,
 'get_company_profile_json': <BB_agent_tool.BB_agent_tool at 0x31f9fb250>,
 'get_equity_shorts_short_interest': <BB_agent_tool.BB_agent_tool at 0x328b1b190>,
 'get_equity_fundamental_historical_splits': <BB_agent_tool.BB_agent_tool at 0x31ee19d50>,
 'get_balance_sheet_json': <BB_agent_tool.BB_agent_tool at 0x31ee390d0>,
 'get_cash_flow_json': <BB_agent_tool.BB_agent_tool at 0x318fb7390>}

In [48]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending on December 31, 2023, was \\$365 million. 

For more details, you can refer to their latest [10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [49]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 333650000000.0, "pe_ratio": 146.64, "foward_pe": 13.3, "eps": 0.9, ...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) is 146.64.

In [50]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 333650000000.0, "pe_ratio": 146.64, "foward_pe": 13.3, "eps": 0.9, ...


The most recent price-to-sales ratio for Merck & Co., Inc. (symbol: MRK) is 5.46.

In [51]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


The most recent revenue per share for Merck & Co Inc (symbol: MRK) is \\$24.25 (TTM: trailing twelve months).

In [52]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# 

In [53]:
# agent_query(f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [54]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Ex-Dividend Date**: March 14, 2024
- **Amount**: \\$0.77 per share
- **Record Date**: March 15, 2024
- **Payment Date**: April 5, 2024
- **Declaration Date**: January 23, 2024

If you need any further details or historical dividend information, feel free to ask!

In [55]:
agent_query(f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 3}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...


It looks like there isn't a directly matching symbol for "MRK." It might be due to an incorrect entry or specific market sector designation. Can you please confirm the company name associated with the "MRK" symbol you are referring to, so I can ensure the correct lookup? For example, "Merck & Co., Inc."

In [56]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-16", "trailing_dividend_yield": 0.0244658856}, {"date": "2023-05-17", "trailing_d...


As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (symbol: MRK) was approximately 2.72%.

In [57]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": 0.0239, "wtd": null, "one_week": 0.0168, "mtd": null, "one_month": 0.0...


The stock performance for Merck & Co., Inc. (symbol: MRK) over the past year has been an increase of 13.2%.

If you need any further details, feel free to ask!

In [58]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [ ]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

In [ ]:
agent_query(f"which ETF has the highest weight in {company} ?", verbose=False)


In [ ]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

In [ ]:
for k, t in BB_agent_tool.agent_registry.items():
    print(t.example_code)

In [ ]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))